In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_rid_of_nan(h):
    for y in range(h.shape[0]):
        for x in range(h.shape[1]):
            if h[y][x] != h[y][x]:
                h[y][x] = 0
    return h

In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
data = np.load('drive/MyDrive/JointStatesX1_2021-05-19_11_44_48_015796_4x4heightmap.npz')
X = data['heightmaps']
X = get_rid_of_nan(X)
y = data['base_links'][:,3:5]
joint_states = data['joint_states']
y = np.hstack((y, joint_states))
print(y.shape)
Xtr,Xts,ytr,yts = train_test_split(X,y, train_size=0.66)
Xtr = torch.from_numpy(Xtr)
ytr = torch.from_numpy(ytr)
Xts = torch.from_numpy(Xts)
yts = torch.from_numpy(yts)
print(Xtr.shape)
print(ytr.shape)

(12, 6)
torch.Size([7, 100])
torch.Size([7, 6])


In [ ]:
from torch import nn
import math

class ConvNeuralNetwork(nn.Module):
    def __init__(self):
        super(ConvNeuralNetwork, self).__init__()
        self.relu = nn.LeakyReLU(0.01, inplace=True)
        self.conv1 = nn.Conv2d(1, 16, [2, 2], bias=False)
        self.conv2 = nn.Conv2d(16, 32, [2, 2], bias=False)
        self.lin1 = nn.Linear(32*8*8, 128)
        self.lin2 = nn.Linear(128, 6)

    def forward(self, input):
        scale = int(math.sqrt(input.shape[0]))
        input = np.reshape(input, (scale,scale))
        input = np.expand_dims(input, 0)
        input = np.expand_dims(input, 0)
        input = torch.from_numpy(input)
        x = self.relu(self.conv1(input))
        x = self.relu(self.conv2(x))
        x = x.view(-1,32*8*8)
        x = self.lin1(x)
        x = self.lin2(x)
        return x

In [ ]:
net = ConvNeuralNetwork()
print(net)

ConvNeuralNetwork(
  (relu): LeakyReLU(negative_slope=0.01, inplace=True)
  (conv1): Conv2d(1, 16, kernel_size=[2, 2], stride=(1, 1), bias=False)
  (conv2): Conv2d(16, 32, kernel_size=[2, 2], stride=(1, 1), bias=False)
  (lin1): Linear(in_features=2048, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=6, bias=True)
)


In [ ]:
#train nn

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

net.train()
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=0)

EPOCH = 4000
step = 0
optimizer.zero_grad()

n = EPOCH * Xtr.shape[0]
losses = np.zeros(n)
steps = np.arange(1,n+1,1)

for epoch in range(EPOCH):
  epoch_train_loss = 0
  for x_,y_ in zip(Xtr,ytr):
    pred = net(x_.float())
    loss = loss_fn(pred[0], y_.float())
    epoch_train_loss += loss.item()
    writer.add_scalar("Loss/train", loss, epoch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    writer.add_scalar('Loss/train', loss.item(), step)
    losses[step] = loss.item()
    step += 1
    #my_func(step,loss.item())

  if (epoch + 1) % 50 == 0:
    print("epoch:",epoch)
    print("loss:",epoch_train_loss)

writer.close()

epoch: 49
loss: 1.7871097577735782
epoch: 99
loss: 1.7325495795812458
epoch: 149
loss: 1.678607332520187
epoch: 199
loss: 1.624599805334583
epoch: 249
loss: 1.5705112933646888
epoch: 299
loss: 1.5162775798235089
epoch: 349
loss: 1.4619113055523485
epoch: 399
loss: 1.4077377442736179
epoch: 449
loss: 1.3539838274009526
epoch: 499
loss: 1.3009104661177844
epoch: 549
loss: 1.2487343992106616
epoch: 599
loss: 1.197820987785235
epoch: 649
loss: 1.1485231986735016
epoch: 699
loss: 1.1012579957023263
epoch: 749
loss: 1.0564419510774314
epoch: 799
loss: 1.0142570564057678
epoch: 849
loss: 0.9750002240762115
epoch: 899
loss: 0.938690546900034
epoch: 949
loss: 0.9053703099489212
epoch: 999
loss: 0.8749307647813112
epoch: 1049
loss: 0.8472211062908173
epoch: 1099
loss: 0.8220148028340191
epoch: 1149
loss: 0.7990222402149811
epoch: 1199
loss: 0.7779483165359125
epoch: 1249
loss: 0.7584848764818162
epoch: 1299
loss: 0.740341630531475
epoch: 1349
loss: 0.7232682943576947
epoch: 1399
loss: 0.70704405

In [ ]:
def predict(X,y,label="data"):
  #conv
  pred = []
  for x in X:
    pred.append(net(x.float()))
  pred = torch.vstack(pred)
  loss = loss_fn(pred, y)
  y_pred = pred.detach().numpy()

  y = y.numpy()
  err = y - y_pred
  err_mean = np.mean(np.abs(err),axis=0)
  value_mean = np.mean(np.abs(y),axis=0)
  print(np.round_(y,2))
  print(np.round_(y_pred,2))
  print("errors mean in rad:",np.round(err_mean, decimals=2))
  print("values mean in rad:",np.round(value_mean, decimals=2))
  print("errors mean:",np.round(np.rad2deg(err_mean), decimals=2))
  print("values mean:",np.round(np.rad2deg(value_mean), decimals=2))
  return err, pred

In [ ]:
e,p = predict(Xtr, ytr, "TRAIN")

[[-0.02 -0.56  2.19  2.19  0.85  0.85]
 [-0.01 -0.02 -0.51 -0.51 -0.14 -0.15]
 [ 0.01  0.36 -2.48 -2.48 -1.22 -1.22]
 [ 0.   -0.02 -0.46 -0.46  0.2   0.2 ]
 [-0.01 -0.46  1.54  1.54  0.42  0.42]
 [-0.01  0.1   0.    0.   -0.41 -0.41]
 [-0.01 -0.15  0.56  0.56  0.3   0.3 ]]
[[-0.01 -0.55  2.22  2.23  0.79  0.79]
 [-0.01  0.   -0.52 -0.51 -0.13 -0.12]
 [ 0.01  0.36 -2.46 -2.46 -1.23 -1.22]
 [-0.    0.02 -0.44 -0.43 -0.06 -0.05]
 [-0.01 -0.31  1.15  1.15  0.37  0.37]
 [-0.   -0.08  0.2   0.18  0.05  0.03]
 [-0.01 -0.23  0.8   0.79  0.29  0.29]]
errors mean in rad: [0.   0.07 0.13 0.13 0.12 0.12]
values mean in rad: [0.01 0.24 1.11 1.11 0.51 0.51]
errors mean: [0.24 4.08 7.44 7.43 7.1  6.88]
values mean: [ 0.49 13.75 63.4  63.4  29.03 29.05]


In [ ]:
e,p = predict(Xts, yts, "TEST")

[[ 0.01  0.02 -1.48 -1.48 -0.21 -0.21]
 [ 0.    0.26  0.    0.   -1.14 -1.14]
 [ 0.   -0.01  0.    0.   -0.14 -0.14]
 [-0.01 -0.02  0.    0.    0.11  0.11]
 [-0.    0.31 -1.5  -1.5  -1.14 -1.14]]
[[-0.    0.36 -2.26 -2.21 -0.77 -0.76]
 [-0.01 -0.01 -0.73 -0.7  -0.41 -0.39]
 [ 0.02  0.54 -2.49 -2.5  -0.77 -0.82]
 [-0.   -0.14  0.44  0.42  0.19  0.17]
 [-0.03 -0.26  0.72  0.73  0.23  0.24]]
errors mean in rad: [0.02 0.37 1.33 1.32 0.67 0.68]
values mean in rad: [0.   0.12 0.6  0.6  0.55 0.55]
errors mean: [ 0.95 21.1  76.41 75.5  38.45 39.17]
values mean: [ 0.22  7.12 34.12 34.12 31.32 31.33]
